# Class Assignment 1: Predicting Medical Cost
#### Aphisit Jaemyaem st126130

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

## 1. Load data

In [3]:
df = pd.read_csv(r"C:\Users\lenovo\OneDrive\Desktop\master\AT82.03 - ML\assigment\insurance.csv")

In [4]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
df.shape

(1338, 7)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.columns

## 2. Exploratory Data Analysis

### 2.1 Univariate analyis

#### Countplot

In [ ]:
sns.countplot(data = df, x = 'smoker')

### Distribution plot

In [ ]:
sns.displot(data = df, x = 'charges')

In [ ]:
sns.displot(data = df, x = 'bmi')

### 2.2 Multivariate analysis

#### Boxplot

In [ ]:
sns.boxplot(x = df["sex"], y = df["bmi"]);
plt.ylabel("Sex")
plt.xlabel("bmi")

#### Scatterplot

In [ ]:
sns.scatterplot(x = df['bmi'], y = df['charges'], hue=df['smoker'])

#### Correlation Matrix

In [ ]:
df = df.drop('region', axis='columns')

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["sex"] = le.fit_transform(df["sex"])

df["sex"].unique()

In [ ]:
le = LabelEncoder()
df["smoker"] = le.fit_transform(df["smoker"])

df["smoker"].unique()

In [ ]:
le.classes_

In [ ]:
le.transform(["no", "yes"])

In [ ]:
plt.figure(figsize = (15,8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm") 

#### 3. Feature selection

In [ ]:
#x is our strong features
X = df[  ['smoker', 'bmi','age','children']  ]

#y is simply the life expectancy col
y = df["charges"]

#### Train test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

#### 4. Preprocessing

In [ ]:
#check for missing values
X_train[['smoker', 'bmi','age','children']].isna().sum()

In [ ]:
X_test[['smoker', 'bmi','age','children']].isna().sum()

In [ ]:
y_train.isna().sum()

In [ ]:
y_test.isna().sum()

In [ ]:
sns.displot(data=df, x='smoker')

In [ ]:
sns.displot(data=df, x='bmi')

In [ ]:
sns.displot(data=df, x='age')

In [ ]:
sns.displot(data=df, x='children')

In [ ]:

sns.displot(y_train)

#### Checking Outliers

In [ ]:
# Create a dictionary of columns.
col_dict = {'smoker':1,'bmi':2,'age':3,'children':4}

# Detect outliers in each variable using box plots.
plt.figure(figsize=(20,30))

for variable,i in col_dict.items():
                     plt.subplot(5,4,i)
                     plt.boxplot(X_train[variable])
                     plt.title(variable)

plt.show()

In [ ]:
def outlier_count(col, data = X_train):
    
    # calculate your 25% quatile and 75% quatile
    q75, q25 = np.percentile(data[col], [75, 25])
    
    # calculate your inter quatile
    iqr = q75 - q25
    
    # min_val and max_val
    min_val = q25 - (iqr*1.5)
    max_val = q75 + (iqr*1.5)
    
    # count number of outliers, which are the data that are less than min_val or more than max_val calculated above
    outlier_count = len(np.where((data[col] > max_val) | (data[col] < min_val))[0])
    
    # calculate the percentage of the outliers
    outlier_percent = round(outlier_count/len(data[col])*100, 2)
    
    if(outlier_count > 0):
        print("\n"+15*'-' + col + 15*'-'+"\n")
        print('Number of outliers: {}'.format(outlier_count))
        print('Percent of data that is outlier: {}%'.format(outlier_percent))

In [ ]:
#check number of outliers for each features.
for col in X_train.columns:
    outlier_count(col)

#### Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

# feature scaling helps improve reach convergence faster
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

#x = (x - mean) / std
#why do we want to scale our data before data analysis / machine learning

#allows your machine learning model to catch the pattern/relationship faster
#faster convergence

#how many ways to scale
#standardardization <====current way
# (x - mean) / std
#--> when your data follows normal distribution

#normalization <---another way
# (x - x_min) / (x_max - x_min)
#---> when your data DOES NOT follow normal distribution (e.g., audio, signal, image)

In [ ]:
# Let's check shapes of all X_train, X_test, y_train, y_test
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of y_test: ", y_test.shape)

#### 5. Modeling

In [ ]:
from sklearn.linear_model import LinearRegression  #we are using regression models
from sklearn.metrics import mean_squared_error, r2_score

lr = LinearRegression()
lr.fit(X_train, y_train)
yhat = lr.predict(X_test)

print("MSE: ", mean_squared_error(y_test, yhat))
print("r2: ", r2_score(y_test, yhat))

#### Much better: Cross validation + Grid search

In [ ]:
from sklearn.linear_model import LinearRegression  #we are using regression models
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Libraries for model evaluation

# models that we will be using, put them in a list
algorithms = [LinearRegression(), SVR(), KNeighborsRegressor(), DecisionTreeRegressor(random_state = 0), 
              RandomForestRegressor(n_estimators = 100, random_state = 0)]

# The names of the models
algorithm_names = ["Linear Regression", "SVR", "KNeighbors Regressor", "Decision-Tree Regressor", "Random-Forest Regressor"]

In [ ]:
y_train.isna().sum()

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

#lists for keeping mse
train_mse = []
test_mse = []

#defining splits
kfold = KFold(n_splits=5, shuffle=True)

for i, model in enumerate(algorithms):
    scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
    print(f"{algorithm_names[i]} - Score: {scores}; Mean: {scores.mean()}")

#### Grid Search

In [ ]:
# use Random-Forest Regressor after i run many model for check MSE and r2.
from sklearn.model_selection import GridSearchCV

param_grid = {'bootstrap': [True], 'max_depth': [5, 10, None],
              'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

rf = RandomForestRegressor(random_state = 1)

grid = GridSearchCV(estimator = rf, 
                    param_grid = param_grid, 
                    cv = kfold, 
                    n_jobs = -1, 
                    return_train_score=True, 
                    refit=True,
                    scoring='neg_mean_squared_error')

# Fit your grid_search
grid.fit(X_train, y_train);

#### 6. Testing

In [ ]:
yhat=grid.predict(X_test)
print("MSE: ", mean_squared_error(y_test, yhat))
print("r2: ", r2_score(y_test, yhat))

#### 7. Analysis: Feature Importance

##### Algorithm way

In [ ]:
rf = grid.best_estimator_

rf.feature_importances_

In [ ]:
#let's plot
plt.barh(X.columns, rf.feature_importances_)

In [ ]:
sorted_idx = rf.feature_importances_.argsort()
plt.barh(X.columns[sorted_idx], rf.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

##### Permutation way

In [ ]:
from sklearn.inspection import permutation_importance

perm_importance = permutation_importance(rf, X_test, y_test)

#let's plot
sorted_idx = perm_importance.importances_mean.argsort()
plt.barh(X.columns[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

##### Shap way

In [ ]:
import shap

explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar", feature_names = X.columns)

#### 8. Inference

In [ ]:
import pickle

# save the model to disk
filename = 'model/Medical Cost.model'
pickle.dump(grid, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
df[['smoker', 'age', 'bmi','children','charges']].loc[1]

In [ ]:
#create unseen value
sample = np.array([[0,18.0000,33.7700,1.0000]])

In [ ]:
#Use this model to predict unseen data set
predicted_life_exp = loaded_model.predict(sample)
predicted_life_exp